In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+En8*ter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gresearch_crypto
from datetime import datetime
import time
import os 

import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt

In [ ]:
def corr(a, b, w):
    cov = lambda x, y: np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    return cov(a, b) / np.sqrt(cov(a, a) * cov(b, b))

# load data & data preprocessing

In [ ]:
data_folder = "/kaggle/input/g-research-crypto-forecasting/"

totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
start_time = totimestamp('01/01/2021')
end_time = totimestamp('21/09/2021')
train_index = slice(start_time,totimestamp('01/07/2021')-60)
test_index = slice(totimestamp('01/07/2021'),end_time)

all_train = pd.read_csv(data_folder+'train.csv').set_index("timestamp").loc[start_time : end_time]

asset_details = pd.read_csv(data_folder+'asset_details.csv')
asset_details['w'] = asset_details['Weight']/asset_details['Weight'].sum()
asset_details = asset_details.set_index('Asset_ID').sort_index()

In [ ]:
all_train

In [ ]:
# pd.pivot_table(all_train[['Asset_ID','Open']],values = 'Open',index =all_train.index ,columns = ['Asset_ID'])


In [ ]:
# del t1

In [ ]:
features = {}
for col in all_train.columns[1:]:
#     features[col] = pd.pivot_table(all_train[['timestamp','Asset_ID',col]],values = col,index ='timestamp' ,columns = ['Asset_ID'])
    features[col] = pd.pivot_table(all_train[['Asset_ID',col]],values = col,index =all_train.index ,columns = ['Asset_ID'])
    features[col] = features[col].reindex(range(start_time,end_time+60,60),method='pad')
    print(f'finish {col}')
    
idx = features['Open'].index
cols = features['Open'].columns

In [ ]:
# source
print('source',features.keys())

# 缺失值比率
print('缺失值比率')
for k,v in features.items():
    print(k,(v.isnull().sum() / v.shape[0]).mean()  )


In [ ]:
features['Open'].head()

缺失值主要因为很多小币在早期没有数据

In [ ]:
# def ts_corr(df):
#     return  np.average(df.corrwith(features['Target']),weights = asset_details['w'] ) 

def corr(a, b, w):
    cov = lambda x, y: np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    return cov(a, b) / np.sqrt(cov(a, a) * cov(b, b))

def log_return(series, periods):
    return np.log(series).diff(periods=periods)

def get_mkt_and_residual_ret(ret):
    market_return = pd.Series(np.nanmean(ret.mul(asset_details['w'],axis=1) ,axis=1),index = idx)
    a = ret.mul(market_return,axis=0).rolling(3750,min_periods=3750).mean()
    b = (market_return*market_return).rolling(3750,min_periods=3750).mean()
    beta = a.div(b,axis=0)
    re_target = ret - beta.mul(market_return,axis=0)
    return market_return,re_target

upper_shadow = (features['High'] - np.maximum(features['Close'],features['Open']))/features['Close']
lower_shadow = (np.minimum(features['Close'],features['Open']) - features['Low'])/features['Close']

logret15 = log_return(features['Close'],15)
logret15_mkt,logret15_re = get_mkt_and_residual_ret(log_return(features['Close'],15))
logret15_mkt= pd.DataFrame(np.tile(logret15_mkt.values,(14,1)).T,index=idx, columns = cols)

logret60 = log_return(features['Close'],60)
logret60_mkt,logret60_re = get_mkt_and_residual_ret(log_return(features['Close'],60))
logret60_mkt= pd.DataFrame(np.tile(logret60_mkt.values,(14,1)).T,index=idx, columns = cols)

logret100 = log_return(features['Close'],100)
logret100_mkt,logret100_re = get_mkt_and_residual_ret(log_return(features['Close'],100))
logret100_mkt= pd.DataFrame(np.tile(logret100_mkt.values,(14,1)).T,index=idx, columns = cols)

In [ ]:
# feats = [(features['High'] - np.maximum(features['Close'],features['Open'])),
#         (np.minimum(features['Close'],features['Open']) - features['Low']),
#          logret15] #改进前的因子

feats = [upper_shadow,lower_shadow,
         logret15_mkt, logret60_mkt, logret100_mkt,
         logret15_re, logret60_re, logret100_re,
        logret15, logret60, logret100]

feats = [feat.unstack() for feat in feats]
X = pd.concat(feats,axis=1)
y = features['Target'].unstack()


X_train = X.loc[(slice(0,13),train_index),:]
y_train = y.loc[slice(0,13),train_index]
X_test = X.loc[(slice(0,13),test_index),:]
y_test = y.loc[slice(0,13),test_index]

# def split(X,y,train_ratio):
#     train_num = int(X.shape[0]*train_ratio)
#     return X.iloc[:train_num],X.iloc[train_num:], y.iloc[:train_num],y.iloc[train_num:]

# X_train, X_valid, y_train, y_valid = split(X_train,y_train,0.8)

In [ ]:
X_train.shape[0]+X_test.shape[0] == X.shape[0]

In [ ]:
y_train.shape[0]+y_test.shape[0] == y.shape[0]

In [ ]:
X.shape[0] == y.shape[0]

# model

## weighter correlation

In [ ]:
def corr(a, b, w):
    cov = lambda x, y: np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    return cov(a, b) / np.sqrt(cov(a, a) * cov(b, b))

In [ ]:
w = pd.Series(index = X_test.index)
for i in asset_details['Weight'].index:
    w.loc[i] = asset_details['Weight'][i]
w

## big model

### linear 

In [ ]:
from sklearn.linear_model import LinearRegression

# implement basic ML baseline (one per asset)
lr = LinearRegression()
lr.fit(X_train.ffill().fillna(0),y_train.ffill().fillna(0))
y_lr_pred = lr.predict(X_test.ffill().fillna(0))
print("y_lr_pred : ",corr(np.nan_to_num(y_lr_pred),np.nan_to_num(y_test.values),np.nan_to_num(np.array(w.to_list()))) )

### lightgbm

In [ ]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=[lgb_train,lgb_eval],
                early_stopping_rounds=50,
               verbose_eval=50)

y_lgb_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print("y_lr_pred : ",corr(np.nan_to_num(y_lgb_pred),np.nan_to_num(y_test.values),np.nan_to_num(np.array(w.to_list()))) )

# small model

In [ ]:
X_trains = {}
y_trains = {}
X_tests = {}
y_tests = {}
for i in range(14):
    X_trains[i] = X_train.loc[(i),:]
    y_trains[i] = y_train.loc[(i),:]
    X_tests[i] = X_test.loc[(i),:]
    y_tests[i] = y_test.loc[(i),:]

In [ ]:
from sklearn.linear_model import LinearRegression

lr_models = {}

def get_lr_model(X_train,y_train):
    lr = LinearRegression()
    lr.fit(X_train.ffill().fillna(0),y_train.ffill().fillna(0))
    return lr

for (i,X_train),(_,y_train) in zip(X_trains.items(),y_trains.items()):
    lr_models[i] = get_lr_model(X_train,y_train)

y_lr_preds = []
for i,lr_model in lr_models.items():
    y_lr_preds.append(lr_model.predict(X_tests[i].ffill().fillna(0)))

print("small seperate model , y_lr_preds : ",corr(np.nan_to_num(np.array(y_lr_preds).flatten()),np.nan_to_num(y_test.values),np.nan_to_num(np.array(w.to_list()))) )

In [ ]:
import lightgbm as lgb
lgb_models = {}
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

def get_lgb_model(X_train,y_train):
    lgb_train = lgb.Dataset(X_train, y_train)
    print('Starting training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    valid_sets=[lgb_train],
                    early_stopping_rounds=50,
                   verbose_eval=50)
    return gbm

for (i,X_train),(_,y_train) in zip(X_trains.items(),y_trains.items()):
    lgb_models[i] = get_lgb_model(X_train,y_train)

y_lgb_preds = []
for i,lgb_model in lgb_models.items():
    y_lgb_preds.append(lgb_model.predict(X_tests[i], num_iteration=lgb_model.best_iteration))
    
print("small seperate model , y_lgb_preds : ",corr(np.nan_to_num(np.array(y_lgb_preds).flatten()),np.nan_to_num(y_test.values),np.nan_to_num(np.array(w.to_list()))) )

In [ ]:
print("small seperate model , y_lgb_preds : ",corr(np.nan_to_num(np.array(y_lgb_preds).flatten()),np.nan_to_num(y_test.values),np.nan_to_num(np.array(w.to_list()))) )

In [ ]:
print("lgb single model for asset 0 ")
np.corrcoef(np.nan_to_num(y_lgb_preds[0]),np.nan_to_num(y_tests[0].values))

In [ ]:
print("linear single model for asset 0 ")
np.corrcoef(np.nan_to_num(y_lr_preds[0]),np.nan_to_num(y_tests[0].values))

# submission

In [ ]:
# import gresearch_crypto
# import traceback

# env = gresearch_crypto.make_env()
# iter_test = env.iter_test()
# # model = lr

# for i, (df_test, df_pred) in enumerate(iter_test):
#     for j , row in df_test.iterrows():        
#         try:
#             print(row)
#             x_test = get_features(row)
#             y_pred = model.predict(x_test.ffill.fillna(0))
#             df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
#         except:
#             df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
#             traceback.print_exc()
    
#     env.predict(df_pred)

### recreating target

In [ ]:
# # r 根据题目定义严格实现 Ra(t)=log(Pa(t+16) / Pa(t+1))
# ret =  np.log(features['Close'].shift(-16)/features['Close'].shift(-1))
# market_return = pd.Series(np.nanmean(ret.mul(asset_details['w'],axis=1)
#                                      ,axis=1),index = idx)

# # beta 根据题目定义实现 beta = m*r / m*m
# a = ret.mul(market_return,axis=0).rolling(3750,min_periods=3750).mean()
# b = (market_return*market_return).rolling(3750,min_periods=3750).mean()
# beta = a.div(b,axis=0)

# #
# re_target = ret - beta.mul(market_return,axis=0)
# print("mean mae",(re_target - features['Target']).abs().mean())
# print("max mae",(re_target - features['Target']).abs().max())
# print("min mae",(re_target - features['Target']).abs().min())

## lgbm tune

In [ ]:
# from sklearn.model_selection import GridSearchCV

# def get_model(X,y):
#     model = LGBMRegressor()
#     model.fit(X, y)
#     return X, y, model

# parameters = {
# #     'max_depth': range (2, 10, 1),
#     'num_leaves': range(21, 161, 10),
#     'learning_rate': [0.1, 0.01, 0.05]
# }

# grid_search = GridSearchCV(
#         estimator=get_model(X_btc_train_scaled,y_btc_train)[2], # bitcoin
#         param_grid=parameters,
#         n_jobs = -1,
#         cv = 5,
#         verbose=True
#     )
# grid_search.fit(X_btc_train_scaled, y_btc_train)
# new_model = grid_search.best_estimator_
# grid_search.best_estimator_

# y_pred_lgbm_tune_btc = new_model.predict(X_btc_test_scaled)
# print('Test score for LGBM tune model: BTC', f"{np.corrcoef(y_pred_lgbm_tune_btc, y_btc_test)[0,1]:.4f}")

## simple nn

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# tf.__version__

# model1 = tf.keras.models.Sequential()
# model1.add(tf.keras.Input(shape=(4,)))
# model1.add(tf.keras.layers.Dense(100, activation='relu'))
# model1.add(tf.keras.layers.Dense(100, activation='relu'))
# model1.add(tf.keras.layers.Dense(100, activation='relu'))
# model1.add(tf.keras.layers.Dense(32, activation='relu'))
# model1.add(tf.keras.layers.Dense(16, activation='relu'))
# model1.add(tf.keras.layers.Dense(1))
# model1.output_shape
# model1.compile(loss='mse',optimizer='adam')
# print(model1.summary())
# model1.fit(X_train, y_train.ffill(), batch_size=3000, epochs=5)

# y_pred = model1.predict(X_valid)
# np.corrcoef(y_pred.flatten(),y_valid.ffill())[0,1]